In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [2]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [17]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([1])


In [18]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [19]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [20]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [21]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.3362, -0.2659,  0.0948, -0.1191, -0.1518,  0.4916,  0.1554, -0.3001,
          0.4366, -0.3788, -0.4759,  0.1912,  0.2784, -0.4120,  0.6482,  0.3936,
          0.3523, -0.1663,  0.4802, -0.2930],
        [-0.0442, -0.6839,  0.2023, -0.2160, -0.2937,  0.4698,  0.3287, -0.0818,
          0.1567,  0.0325, -0.4593, -0.2266,  0.4768, -0.0835,  0.2360,  0.1293,
          0.4604, -0.1669,  0.4921, -0.2023],
        [-0.3536, -0.0475, -0.1239, -0.4441,  0.1171,  0.1996,  0.3329, -0.4055,
          0.2578, -0.3536, -0.0612, -0.1838,  0.6516, -0.3232,  0.3558,  0.1649,
          0.1071,  0.0513,  0.3084, -0.1730]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.0948, 0.0000, 0.0000, 0.4916, 0.1554, 0.0000, 0.4366,
         0.0000, 0.0000, 0.1912, 0.2784, 0.0000, 0.6482, 0.3936, 0.3523, 0.0000,
         0.4802, 0.0000],
        [0.0000, 0.0000, 0.2023, 0.0000, 0.0000, 0.4698, 0.3287, 0.0000, 0.1567,
         0.0325, 0.0000, 0.0000, 0.4768, 0.0000, 0.23

In [23]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)
print(logits)

tensor([[-0.1638,  0.0383, -0.0332,  0.3281,  0.2574,  0.2049,  0.1877,  0.5655,
         -0.0545, -0.1663],
        [-0.1998, -0.1102,  0.1622,  0.3658,  0.1178,  0.1197,  0.1440,  0.3352,
          0.0012, -0.0572],
        [-0.2425,  0.0128, -0.0330,  0.3182,  0.1259,  0.1594,  0.0935,  0.4460,
         -0.0427, -0.0886]], grad_fn=<AddmmBackward0>)


In [25]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(pred_probab)

tensor([[0.0737, 0.0902, 0.0839, 0.1205, 0.1122, 0.1065, 0.1047, 0.1527, 0.0822,
         0.0735],
        [0.0739, 0.0808, 0.1061, 0.1301, 0.1015, 0.1017, 0.1042, 0.1262, 0.0903,
         0.0852],
        [0.0715, 0.0922, 0.0881, 0.1252, 0.1033, 0.1068, 0.1000, 0.1423, 0.0873,
         0.0834]], grad_fn=<SoftmaxBackward0>)


In [26]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0322,  0.0030,  0.0204,  ..., -0.0219, -0.0331,  0.0133],
        [ 0.0314, -0.0185, -0.0179,  ..., -0.0031, -0.0093, -0.0270]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0201,  0.0083], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0304,  0.0062,  0.0398,  ..., -0.0073, -0.0192, -0.0020],
        [-0.0434, -0.0328,  0.0426,  ...,  0.0378, -0.0069,  0.0015]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | 